### Data Addition
Compile all additions to the data here

Update cleaned_BankChurner (or whatever the final dataset name will be)

In [1]:
# libraries
import os
import numpy as np
import pandas as pd
from sklearn.neighbors import KernelDensity

In [7]:
# set path
parent_directory = os.path.abspath(os.path.join('..'))
PATH = os.path.join(parent_directory, 'data')

# helper functions
def load_data(path):
    return pd.read_csv(os.path.join(PATH, path))

In [ ]:
# load main dataset
bank_churners = load_data('BankChurners.csv') # main dataset

In [ ]:
# clean bank_churners

### Data Addition:

Add dataset name, link to dataset, and other relevant details

##### Subgroup B Q1: InsuranceProducts

Link: https://www.kaggle.com/datasets/akhilups/insurance-product-purchase-prediction?resource=download

**Product Options** ( 'A', 'B', 'C', 'D', 'E', 'F', 'G')

Each product has 7 customizable options selected by customers, each with 2, 3, or 4 ordinal values possible:

A product is simply a vector with length 7 whose values are chosen from each of the options listed above. The cost of a product is a function of both the product options and customer characteristics.

Columns Added: 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'risk_factor'

In [9]:
def generate_new_data(df_target, df_source, colname, lowerlim=None, upperlim=None, kernel='gaussian',  inttype=False):
    kde_new = KernelDensity(bandwidth=1, kernel=kernel)
    kde_new.fit(df_source[colname].values.reshape(-1, 1))

    new_samples = kde_new.sample(len(df_target)).reshape(-1)        
    df_target[colname] = new_samples

    if lowerlim != None:
        df_target.loc[df_target[colname] < lowerlim, colname] = lowerlim
    if upperlim != None:
        df_target.loc[df_target[colname] > upperlim, colname] = upperlim
    
    return df_target

In [ ]:
# load datasets
insurance_products = load_data('InsuranceProduct.csv') # dataset with different products and services

# merge datasets
df = bank_churners.drop(['Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_1', 
                         'Naive_Bayes_Classifier_Attrition_Flag_Card_Category_Contacts_Count_12_mon_Dependent_count_Education_Level_Months_Inactive_12_mon_2'],
                         axis=1) 	

# fill na values for risk factor with median and 0 for everything else
cols_to_merge = ['A', 'B', 'C', 'D', 'E', 'F', 'G', 'risk_factor']
insurance_products['risk_factor'] = insurance_products['risk_factor'].fillna(insurance_products['risk_factor'].median()) 
insurance_products = insurance_products[cols_to_merge].fillna(0)

col_limits = {'A': [0, 2], 'B': [0, 1], 'C': [1, 4], 'D': [1, 3], 'E': [0, 1], 'F': [0, 3], 'G': [1, 4], 'risk_factor': [0, None]}
for col in cols_to_merge:
    df = generate_new_data(df, insurance_products, col, lowerlim=col_limits[col][0], upperlim=col_limits[col][1])
    if col != 'cost':
        df[col] = np.round(df[col]).astype(int)